In [1]:
import os
import pandas as pd

In [2]:
data_path = 'C:/Users\lukec\OneDrive - University of Cambridge\PhD\Data\ClimateTRACE\May23/'
folders = ['manufacturing/']#, 'mineral_extraction/', 'fossil_fuel_operations/', 'power/', 'waste/']
save_path = "C:/Users\lukec\PycharmProjects\emissions-tracking-conda\emissions-tracking\data\classification_inputs/CT_manufacturing"
key_words = ['asset', 'emissions']

files = []
for folder in folders:
    folder_files = os.listdir(data_path+folder)
    files = files + [data_path+folder+file for file in folder_files if key_words[0] in file and key_words[1] in file and 'transport' not in file]

In [8]:
def filter_df(df:pd.DataFrame, cols:list, filters:list) -> pd.DataFrame:
    """Function for filtering a pandas dataframe based on column values"""
    for col, filt in zip(cols, filters):
        df = df[[i in filt for i in df[col]]]
    return df

In [9]:
data = pd.DataFrame()
for file in files:
    data = pd.concat((data, pd.read_csv(file)))

data_used = filter_df(data, ['gas'], [['co2e_100yr']])

# Extract lat/longs
data_used['LONGITUDE'] = [i.strip("POINT()").split(" ")[0] for i in data_used['st_astext']]
data_used['LATITUDE'] = [i.strip("POINT()").split(" ")[1] for i in data_used['st_astext']]

# Extract dates
data_used['Year'] = pd.to_datetime(data_used['start_time']).dt.year
data_used['Month'] = pd.to_datetime(data_used['start_time']).dt.month

columns = ['asset_id', 'iso3_country', 'original_inventory_sector', 'capacity', 'asset_name', 'asset_type', 'LONGITUDE', 'LATITUDE', 'Year', 'Month', 'emissions_quantity']

data_used = data_used[columns].rename(columns={'emissions_quantity':'Emissions'}).dropna(subset=['Emissions'])

C:\Users\lukec\AppData\Local\Temp/ipykernel_25480/2374859088.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_used['LONGITUDE'] = [i.strip("POINT()").split(" ")[0] for i in data_used['st_astext']]
C:\Users\lukec\AppData\Local\Temp/ipykernel_25480/2374859088.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_used['LATITUDE'] = [i.strip("POINT()").split(" ")[1] for i in data_used['st_astext']]
C:\Users\lukec\AppData\Local\Temp/ipykernel_25480/2374859088.py:12: SettingWithCopyWarning: 
A value is

In [10]:
data_used = data_used.merge(data_used[['Year', 'Month']].drop_duplicates().sort_values(['Year', 'Month']).reset_index(drop=True).reset_index().rename(columns={'index':'Timestep'}), on=['Year', 'Month'])
data_used = data_used.drop(columns=['Year', 'Month'])

In [6]:
data_used

,asset_id,iso3_country,original_inventory_sector,capacity,asset_name,asset_type,LONGITUDE,LATITUDE,Emissions,Timestep
0,22087016,IND,aluminum,410000.0,Korba aluminium plant,Unknown,82.736427,22.395148,60175.0,52
1,22087017,IND,aluminum,360000.0,Lapanga aluminium plant,Unknown,84.048315,21.739215,52837.0,52
2,22087018,IND,aluminum,345000.0,Renukoot aluminium plant,Unknown,83.030123,24.223115,50635.0,52
3,22087019,IRN,aluminum,60000.0,Arak aluminium plant,Unknown,49.758279,34.076901,8806.0,52
4,22087020,IRN,aluminum,105000.0,Arak (2) aluminium plant,Unknown,49.76274,34.078385,15411.0,52
...,...,...,...,...,...,...,...,...,...,...
291147,2823,IDN,steel,2500000.0,Krakatau Steel plant,integrated (DRI),106.001766,-6.007021,50417.0,89
291148,2376,AUS,steel,3200000.0,BlueScope Port Kembla steel plant,integrated (BF),150.886191,-34.463834,372552.0,89
291149,2477,CHN,steel,6500000.0,"Tangshan Ganglu Iron and Steel Co., Ltd.",integrated (BF),118.073913,40.183913,846930.0,89
291150,2658,CHN,steel,7300000.0,"Baosteel Group Xinjiang Bayi Iron & Steel Co.,...",integrated (BF),87.304639,43.852147,632478.0,89


In [7]:
feature_cols = [i for i in data_used.columns if i not in ['Timestep', 'Emissions']]
# data_pivoted = data_used.pivot(index=feature_cols,
#                       columns=['Year', 'Month'] if 'Month' in data_used.columns else 'Year',
#                       values = 'Emissions').reset_index()
data_pivoted = data_used.pivot(index=feature_cols, columns='Timestep', values = 'Emissions').reset_index()

In [8]:
data_pivoted

Timestep,asset_id,iso3_country,original_inventory_sector,capacity,asset_name,asset_type,LONGITUDE,LATITUDE,0,1,...,80,81,82,83,84,85,86,87,88,89
0,2368,ARE,steel,3500000.0,GHC Emirates Steel Industries Abu Dhabi plant,integrated (DRI),54.467987,24.322033,62492.0,62492.0,...,41140.0,49744.0,59358.0,62084.0,44916.0,46388.0,61382.0,67847.0,70583.0,67945.0
1,2369,ALB,steel,700000.0,Kurum International Elbasan steel plant,integrated (BF),20.026117,41.094611,21088.0,21088.0,...,6808.0,6808.0,6808.0,6808.0,6808.0,6808.0,6808.0,6808.0,6808.0,6808.0
2,2370,ARG,steel,1700000.0,ArcelorMittal Acindar Villa Constitución steel...,integrated (DRI),-60.300415,-33.249936,28164.0,26884.0,...,25371.0,30554.0,32351.0,25523.0,23674.0,23793.0,28470.0,19893.0,29533.0,31276.0
3,2371,ARG,steel,1300000.0,TenarisSiderca Campana steel plant,integrated (DRI),-58.977388,-34.152554,21537.0,20558.0,...,19402.0,23365.0,24739.0,19518.0,18103.0,18195.0,21771.0,15212.0,22584.0,23917.0
4,2372,ARG,steel,3200000.0,Ternium Siderar San Nicolás steel plant,integrated (BF),-60.138697,-33.37365,317156.0,317156.0,...,308826.0,253270.0,230394.0,300656.0,207518.0,148694.0,235296.0,267976.0,267976.0,267976.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3290,22087062,USA,aluminum,220000.0,Sebree aluminium plant,Unknown,-87.500499,37.657434,66114.0,59761.0,...,52205.0,54609.0,52892.0,55296.0,NaN,NaN,NaN,NaN,NaN,NaN
3291,22087063,USA,aluminum,187000.0,New Madrid aluminium plant,Unknown,-89.56376,36.508977,56197.0,50796.0,...,44374.0,46417.0,44958.0,47001.0,NaN,NaN,NaN,NaN,NaN,NaN
3292,22087064,USA,aluminum,135000.0,Massena aluminium plant,Unknown,-74.89016,44.961979,40570.0,36671.0,...,32035.0,33510.0,32456.0,33931.0,NaN,NaN,NaN,NaN,NaN,NaN
3293,22087065,USA,aluminum,184000.0,Mount Holly aluminium plant,Unknown,-80.054213,33.049111,55296.0,49982.0,...,43662.0,45673.0,44237.0,46247.0,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
data_pivoted.to_csv(save_path+'.csv', index=False)
data_used.to_csv(save_path+'_melted.csv', index=False)

In [3]:
data = pd.read_csv(save_path+'.csv')

In [7]:
data

,asset_id,iso3_country,original_inventory_sector,capacity,asset_name,asset_type,LONGITUDE,LATITUDE,0,1,...,80,81,82,83,84,85,86,87,88,89
0,2368,ARE,steel,3500000.0,GHC Emirates Steel Industries Abu Dhabi plant,integrated (DRI),54.467987,24.322033,62492.0,62492.0,...,41140.0,49744.0,59358.0,62084.0,44916.0,46388.0,61382.0,67847.0,70583.0,67945.0
1,2369,ALB,steel,700000.0,Kurum International Elbasan steel plant,integrated (BF),20.026117,41.094611,21088.0,21088.0,...,6808.0,6808.0,6808.0,6808.0,6808.0,6808.0,6808.0,6808.0,6808.0,6808.0
2,2370,ARG,steel,1700000.0,ArcelorMittal Acindar Villa Constitución steel...,integrated (DRI),-60.300415,-33.249936,28164.0,26884.0,...,25371.0,30554.0,32351.0,25523.0,23674.0,23793.0,28470.0,19893.0,29533.0,31276.0
3,2371,ARG,steel,1300000.0,TenarisSiderca Campana steel plant,integrated (DRI),-58.977388,-34.152554,21537.0,20558.0,...,19402.0,23365.0,24739.0,19518.0,18103.0,18195.0,21771.0,15212.0,22584.0,23917.0
4,2372,ARG,steel,3200000.0,Ternium Siderar San Nicolás steel plant,integrated (BF),-60.138697,-33.373650,317156.0,317156.0,...,308826.0,253270.0,230394.0,300656.0,207518.0,148694.0,235296.0,267976.0,267976.0,267976.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3290,22087062,USA,aluminum,220000.0,Sebree aluminium plant,Unknown,-87.500499,37.657434,66114.0,59761.0,...,52205.0,54609.0,52892.0,55296.0,NaN,NaN,NaN,NaN,NaN,NaN
3291,22087063,USA,aluminum,187000.0,New Madrid aluminium plant,Unknown,-89.563760,36.508977,56197.0,50796.0,...,44374.0,46417.0,44958.0,47001.0,NaN,NaN,NaN,NaN,NaN,NaN
3292,22087064,USA,aluminum,135000.0,Massena aluminium plant,Unknown,-74.890160,44.961979,40570.0,36671.0,...,32035.0,33510.0,32456.0,33931.0,NaN,NaN,NaN,NaN,NaN,NaN
3293,22087065,USA,aluminum,184000.0,Mount Holly aluminium plant,Unknown,-80.054213,33.049111,55296.0,49982.0,...,43662.0,45673.0,44237.0,46247.0,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
data['capacity'].unique()

array([ 3500000.    ,   700000.    ,  1700000.    ,  1300000.    ,
        3200000.    ,   650000.    ,  6000000.    ,  1570000.    ,
        1200000.    ,   790000.    ,   620000.    ,  1000000.    ,
        1940000.    ,  1008000.    ,  1400000.    ,   580000.    ,
         800000.    ,  5000000.    ,   850000.    ,  4500000.    ,
        7500000.    ,  6250000.    ,  3000000.    ,  1100000.    ,
        3620000.    ,   600000.    ,   950000.    ,   540000.    ,
         500000.    ,   900000.    ,  2400000.    ,  4050000.    ,
        2500000.    ,   949000.    ,   668000.    ,  1450000.    ,
         520000.    , 17600000.    ,  3800000.    ,  1260000.    ,
        2000000.    ,  8400000.    ,  6800000.    ,  2550000.    ,
        4620000.    ,  1150000.    ,  1800000.    ,   720000.    ,
        6500000.    ,  4000000.    ,  8000000.    , 12528000.    ,
        3220000.    ,  1610000.    , 12500000.    ,  3400000.    ,
        3335000.    ,  2070000.    ,  1600000.    , 13000000. 